# Housing Price Regression Model

## Introduction & Problem Statement
---

For many people, a home represents a sizable portion of their assets and net worth. Moving houses can also be a lengthy and expensive process. It is for this reason that we founded RENO-BOT, the leading AI Renovation Advisor start-up, and it is our mission to help homeowners maximize the value of their houses in the event they wish to move. 

As Co-Founder and Chief Data Scientist, it is my pleasure to walk you through our proprietary housing price regression model that is the backbone of RENO-BOT.

In housing price regression model, we seek to **predict house sales prices with linear regression models**, to answer the question, **which factors influence house sales prices the most?**

Through the linear regression model, we can quantify the impact that key factors have on house prices. Some factors cannot be changed like location, but others could be. This would be informative to house owners who are interested in making improvements to their home, as reference point to evaluate cost benefit of potential improvements. 

Our data is based on the housing dataset from Ames, Iowa, and contains over 80 features. In order that our model covers comprehensively all aspects of a house, similar housing features will be grouped according to key components, and features will be selected from within each component group using pairwise correlation. This would then be aggregated into a regression model.

Its a common saying that location is most important in buying a house. So we would group selected features to run the regression model, first area as baseline, then area plus location features, then area plus house features, then lastly all features together.

http://jse.amstat.org/v19n3/decock/DataDocumentation.txt

## 1 Data Preprocessing & Cleaning
---

### 1.1 Loading Libraries & Data


In [1]:
# Importing libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import (LinearRegression, LogisticRegressionCV, Lasso, LassoCV, 
Ridge, RidgeCV, ElasticNet, ElasticNetCV)
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
import scipy
import scipy.cluster.hierarchy as sch
import scipy.stats as ss

C:\Users\deang\anaconda3\envs\dsi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\deang\anaconda3\envs\dsi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
# Load dataset
df_train = pd.read_csv('../project_2/datasets/train.csv')
df_test = pd.read_csv('../project_2/datasets/test.csv')

# 1.2 Data structure
---

In this section, the shape, column names and data types of the ```test``` and ```train``` data sets would be compared. This is to help us to understand how we can concatenate the two data sets into one, for the purposes of more efficient data cleaning and feature engineering.

We do expect that there is test set lacks the last column, ```SalePrice```, which is what we are setting out to predict in this project.

In [3]:
# Check shape of Train dataset
df_train.shape

(2051, 81)

In [4]:
# Check shape of Test dataset
df_test.shape

(878, 80)

The columns ```Id``` and ```PID``` will be dropped at the start as these are unique identifiers of each property and are not useful for our regression study, hence to be dropped to reduce noise in the model.

In [5]:
df_train.drop(columns=['Id','PID'], inplace=True)
df_test.drop(columns=['Id','PID'], inplace=True)

In [6]:
# Check if column names are same, excluing the last column, SalePrice
df_train.columns[:-1] == df_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [7]:
# Check if data types are same, exclduing the last column, SalePrice
dtype_check = df_train.dtypes[:-1] == df_test.dtypes

# Create mask of columns that did not match, and display those rows 
dtype_check[dtype_check==False].index
display(df_train[dtype_check[dtype_check==False].index].dtypes)
display(df_test[dtype_check[dtype_check==False].index].dtypes)

# For purposes of this project, its not an issue that these columns are float or int as they 
# are still numeric datatypes

BsmtFin SF 1      float64
BsmtFin SF 2      float64
Bsmt Unf SF       float64
Total Bsmt SF     float64
Bsmt Full Bath    float64
Bsmt Half Bath    float64
Garage Cars       float64
Garage Area       float64
dtype: object

BsmtFin SF 1      int64
BsmtFin SF 2      int64
Bsmt Unf SF       int64
Total Bsmt SF     int64
Bsmt Full Bath    int64
Bsmt Half Bath    int64
Garage Cars       int64
Garage Area       int64
dtype: object

In [8]:
# Concatenate train and test datasets
df_combine = pd.concat([df_train, df_test], keys=['train','test'], sort=False)

In [9]:
# set names of indexes
df_combine.index.set_names(['dataset','index']);

In [10]:
# Create masks for categorical and numerical features
category_columns = [key for key, val in df_train.iteritems() 
                    if df_train.dtypes[key] == 'object']
numerical_columns = [key for key, val in df_train.iteritems() 
                     if (df_train.dtypes[key] == 'int64') or (df_train.dtypes[key] == 'float64')]
print(len(category_columns))
print(len(numerical_columns))

42
37


Columns are grouped by key component features of a house, in the dictionary below to facilitate analysis

In [11]:
# Retrieve column names for sorting
df_combine.columns

Index(['MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area', 'Street',
       'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config',
       'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Gara

In [12]:
# Sort and group columns by key component features
col_group = {'location': ['MS Zoning', 'Neighborhood', 'Condition 1', 'Condition 2',], 
             'land': ['Lot Frontage', 'Lot Area', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 
                      'Lot Config', 'Land Slope'], 
             'structure': ['MS SubClass', 'Utilities', 'Bldg Type', 'House Style',  'Roof Style', 
                           'Foundation',  'Heating', 'Heating QC', 'Central Air', 'Electrical', 
                           'Fireplaces', 'Fireplace Qu','Functional'], 
             'finishings': ['Overall Qual', 'Overall Cond', 'Roof Matl', 'Exterior 1st', 
                            'Exterior 2nd', 'Mas Vnr Type','Mas Vnr Area', 
                            'Exter Qual', 'Exter Cond'], 
             'liv_area': ['1st Flr SF','2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath', 
                          'Half Bath', 'Bedroom AbvGr','Kitchen AbvGr', 'Kitchen Qual', 
                          'TotRms AbvGrd',],
             'basement': ['Bsmt Qual','Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
                          'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
                          'Bsmt Full Bath','Bsmt Half Bath',], 
             'garage': ['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars', 
                        'Garage Area', 'Garage Qual','Garage Cond'], 
             'external': ['Paved Drive', 'Wood Deck SF', 'Open Porch SF','Enclosed Porch', 
                          '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Pool QC','Fence', 
                          'Misc Feature', 'Misc Val'], 
             'age_sale': ['Year Built', 'Year Remod/Add', 'Mo Sold', 'Yr Sold', 'Sale Type', 
                          'SalePrice']}

In [13]:
# Check total number of items in col_group dict
x = 0
for items in col_group.values():
    x = x + len(items)
x == len(df_combine.columns)

True

## 1.3 Data Cleaning
---

### 1.3.1 Investigate Null/Zero Values

In this section, null & zero values will be investigated. 

Null & zero values will first be checked against the original data documentation to check if there was intent to have null categories and zero values in each feature. 

Then we will filter out rows with null/zeroes within each category, and compare across feature columns. 

If all columns are null/zero then we have greater certainty that data is not missing. If there are data in other rows that could indicate that null/zero data is missing data, then further processing would be considered.

Changes to data which are more specific in nature would be carried out within each section, except for converting of ```NaN``` to ```'none'``` or ```0``` which would be done wholesale at the end of the section.

In [14]:
# Generate count of null values by column
null_series=df_combine.isnull().sum()
display(null_series[null_series>0])
null_cols = null_series[null_series>0].index.tolist()

Lot Frontage       490
Alley             2731
Mas Vnr Type        23
Mas Vnr Area        23
Bsmt Qual           80
Bsmt Cond           80
Bsmt Exposure       83
BsmtFin Type 1      80
BsmtFin SF 1         1
BsmtFin Type 2      81
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
Electrical           1
Bsmt Full Bath       2
Bsmt Half Bath       2
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual        159
Garage Cond        159
Pool QC           2916
Fence             2357
Misc Feature      2823
SalePrice          878
dtype: int64

##### 1.3.1.1 Land Features

In [15]:
df_combine[df_combine[col_group['land']].isna().any(axis=1)][col_group['land']].head()

Lot Frontage  Lot Area Street Alley Lot Shape Land Contour  \
train 0           NaN     13517   Pave   NaN       IR1          Lvl   
      1          43.0     11492   Pave   NaN       IR1          Lvl   
      2          68.0      7922   Pave   NaN       Reg          Lvl   
      3          73.0      9802   Pave   NaN       Reg          Lvl   
      4          82.0     14235   Pave   NaN       IR1          Lvl   

        Lot Config Land Slope  
train 0    CulDSac        Gtl  
      1    CulDSac        Gtl  
      2     Inside        Gtl  
      3     Inside        Gtl  
      4     Inside        Gtl

**Lot Frontage**
As defined in the data documentation, ```Lot Frontage``` is the linear feet of street connected to property. 

Intuitively, the length of lot frontage would be correlated with the area of the plot. But the shape of the plot matters also. From the Land Features grouping, it appears that ```Lot Shape``` and ```Lot Config``` might be a lot shape related feature. 

We thus plan to impute ```NaN``` values of ```Lot Frontage``` with the ratio of ```Lot Frontage``` to ```Lot Area```, based on average ratio by lot shape.

To decide which lot shape to use, descriptive statistics would be calculated by grouping each category in ```Lot Shape``` and ```Lot Config```, and the feature with categories that have lower standard deviation would be chosen.

In [16]:
# Calculate Lot Frontage Ratio
df_combine['lot_frontage_ratio'] = df_combine['Lot Frontage'] / df_combine['Lot Area']

In [17]:
# Descriptive statsitics lot frontage ratio for Lot Shape
df_combine.groupby('Lot Shape')['lot_frontage_ratio'].agg(['count', 'mean', 'std'])

,count,mean,std
Lot Shape,,,
IR1,653,0.007427,0.002597
IR2,48,0.005276,0.003363
IR3,11,0.005552,0.003657
Reg,1727,0.008205,0.002184


In [18]:
# Descriptive statsitics lot frontage ratio for Lot Config
df_combine.groupby('Lot Config')['lot_frontage_ratio'].agg(['count', 'mean', 'std'])

,count,mean,std
Lot Config,,,
Corner,407,0.008498,0.002394
CulDSac,92,0.004779,0.002598
FR2,65,0.007026,0.002132
FR3,10,0.008958,0.002955
Inside,1865,0.007984,0.002266


As Lot Config has lower standard deviation of lot frontage ratio per category, we would use this to impute lot frontage for missing values.

In [19]:
# Calculate ratio of Lot Frontage to Lot Area by Lot Config
df_train_temp = df_combine.loc['train',:]
df_train_temp['lot_frontage_ratio'] = df_train_temp.loc[:,'Lot Frontage'] /df_train_temp.loc[:,'Lot Area']
display(df_train_temp.groupby('Lot Config')['lot_frontage_ratio'].describe())
ratio_by_config = df_train_temp.groupby('Lot Config')['lot_frontage_ratio'].mean()

C:\Users\deang\AppData\Local\Temp\ipykernel_10348\1765268012.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_temp['lot_frontage_ratio'] = df_train_temp.loc[:,'Lot Frontage'] /df_train_temp.loc[:,'Lot Area']


,count,mean,std,min,25%,50%,75%,max
Lot Config,,,,,,,,
Corner,281.0,0.008506,0.002622,0.004438,0.007272,0.008195,0.009154,0.037154
CulDSac,70.0,0.004403,0.002101,0.000878,0.003149,0.003782,0.005159,0.009964
FR2,49.0,0.006893,0.002271,0.002376,0.005282,0.007111,0.008299,0.010526
FR3,8.0,0.008871,0.003344,0.005714,0.007534,0.007844,0.008668,0.016667
Inside,1313.0,0.008013,0.002239,0.000876,0.006667,0.007812,0.008696,0.022708


In [20]:
# Impute Lot Frontage from ratio of Lot Frontage and lot Area
df_combine['lot_frontage_ratio'].fillna(df_combine['Lot Config'].map(ratio_by_config), inplace=True)
df_combine['lot_frontage_est'] = df_combine['Lot Area'] * df_combine['lot_frontage_ratio']
df_combine.drop(columns='Lot Frontage',inplace=True)
df_combine.head()
#def convert_lot_frontage(df):

MS SubClass MS Zoning  Lot Area Street Alley Lot Shape Land Contour  \
train 0           60        RL     13517   Pave   NaN       IR1          Lvl   
      1           60        RL     11492   Pave   NaN       IR1          Lvl   
      2           20        RL      7922   Pave   NaN       Reg          Lvl   
      3           60        RL      9802   Pave   NaN       Reg          Lvl   
      4           50        RL     14235   Pave   NaN       IR1          Lvl   

        Utilities Lot Config Land Slope  ... Pool QC Fence Misc Feature  \
train 0    AllPub    CulDSac        Gtl  ...     NaN   NaN          NaN   
      1    AllPub    CulDSac        Gtl  ...     NaN   NaN          NaN   
      2    AllPub     Inside        Gtl  ...     NaN   NaN          NaN   
      3    AllPub     Inside        Gtl  ...     NaN   NaN          NaN   
      4    AllPub     Inside        Gtl  ...     NaN   NaN          NaN   

        Misc Val Mo Sold  Yr Sold  Sale Type  SalePrice  lot_frontage_ratio  \
train 0        0       3     2010        WD    130500.0            0.004403   
      1        0       4     2009        WD    220000.0            0.003742   
      2        0       1     2010        WD    109000.0            0.008584   
      3        0       4     2010        WD    174000.0            0.007447   
      4        0       3     2010        WD    138500.0            0.005760   

        lot_frontage_est  
train 0        59.516825  
      1        43.000000  
      2        68.000000  
      3        73.000000  
      4        82.000000  

[5 rows x 80 columns]

**Alley**

According to the data documentation, ```NaN``` values indicate no alley access, and with no other category that seems related to this, we would accept this as is, and ```NaN``` would be replaced with the string ```none```.

##### 1.3.1.2 Structural Features

As seen in the table below, all values in ```Fireplaces``` that are zero correspons to ```NaN``` in ```Fireplace Qu``` (Fireplace Quality), hence we can be more certain that there is no missing data here. ```NaN``` would be replaced with the string ```none```.

In [21]:
df_combine[df_combine[col_group['structure']].isna().any(axis=1)][col_group['structure']]
df_combine.groupby(['Fireplaces','Fireplace Qu'],dropna=False
                  ).size().reset_index().rename(columns={0:'count'})

,Fireplaces,Fireplace Qu,count
0,0,NaN,1422
1,1,Ex,37
2,1,Fa,64
3,1,Gd,626
4,1,Po,46
5,1,TA,500
6,2,Ex,5
7,2,Fa,10
8,2,Gd,112
9,2,TA,94


##### 1.3.1.3 Finishing Features

```Mas Vnr Type``` is compared with ```Mas Vnr Area```, and we see that all ```NaN``` values of both features corresponds to each other only, hence  ```NaN``` would be replaced with the string ```none```.

In [22]:
df_combine[df_combine[col_group['finishings']].isna().any(axis=1)][col_group['finishings']]
df_combine.groupby([ 'Mas Vnr Type','Mas Vnr Area'],dropna=False
                  ).size().reset_index().rename(columns={0:'count'})

,Mas Vnr Type,Mas Vnr Area,count
0,BrkCmn,40.0,1
1,BrkCmn,41.0,1
2,BrkCmn,54.0,1
3,BrkCmn,58.0,2
4,BrkCmn,66.0,1
...,...,...,...
588,Stone,796.0,1
589,Stone,860.0,1
590,Stone,902.0,1
591,Stone,1224.0,1


##### 1.3.1.4 Basement Features

A temporary dataframe would be created and filtered for summed total square footage

In [23]:
#select rows with null values
df_basement_nan = df_combine[df_combine[col_group['basement']]
                             .isna().any(axis=1)][col_group['basement']]
#sum measures of area
df_basement_nan['sumsqft'] = df_basement_nan['BsmtFin SF 1'] + df_basement_nan['BsmtFin SF 2'] + \
                            df_basement_nan['Total Bsmt SF']
#select rows where sum of area measures is not equal to zero to validate that there is no basement
df_basement_nan[df_basement_nan['sumsqft'] != 0]

Bsmt Qual Bsmt Cond Bsmt Exposure BsmtFin Type 1  BsmtFin SF 1  \
train 1147        Gd        TA            No            GLQ        1124.0   
      1327       NaN       NaN           NaN            NaN           NaN   
      1456        Gd        TA           NaN            Unf           0.0   
      1547        Gd        TA           NaN            Unf           0.0   
      1997        Gd        TA           NaN            Unf           0.0   

           BsmtFin Type 2  BsmtFin SF 2  Bsmt Unf SF  Total Bsmt SF  \
train 1147            NaN         479.0       1603.0         3206.0   
      1327            NaN           NaN          NaN            NaN   
      1456            Unf           0.0        725.0          725.0   
      1547            Unf           0.0       1595.0         1595.0   
      1997            Unf           0.0        936.0          936.0   

            Bsmt Full Bath  Bsmt Half Bath  sumsqft  
train 1147             1.0             0.0   4809.0  
      1327             NaN             NaN      NaN  
      1456             0.0             0.0    725.0  
      1547             0.0             0.0   1595.0  
      1997             0.0             0.0    936.0

Issue 1:

We observe that there is zero data for SF 1 & 2 data for Type 1 & 2```Unf``` (unfinished category) ```BsmtFin SF 1``` and ```BsmtFin SF 2``` when there is values in the ```Bsmt Unf SF``` which is the total for unfinished square foot area.

Hence we would rely on just the ```Bsmt Unf SF``` row and drop the ```BsmtFin SF 1``` and ```BsmtFin SF 2```. Additionally we would calculate the finished area and drop ```Total Bsmt SF``` to reduce overlap in data.

In [24]:
def calculate_basement_fin_sf(df):
    '''calculates finished square foot area of basement from total and unfinished 
    and drops total and SF 1 & 2 '''
    df['bsmt_fin_sf'] = df['Total Bsmt SF'] - df['Bsmt Unf SF']
    df.drop(labels = ['Total Bsmt SF','BsmtFin SF 1','BsmtFin SF 2'], axis=1, inplace=True)
calculate_basement_fin_sf(df_combine)

Issue 2: 

For item 1147 seen in the table above, we can deduce that ```BsmtFin SF 2``` is not of the the unfinished basement type, based on subtracting ```BsmtFin Type 1``` & ```Bsmt Unf SF``` from ```Total Bsmt SF```.
We would assign the value 'Rec', as to the most common category for those with GLQ as type_1 after 'Unf'.

In [25]:
# Count of rows by combination of categories in BsmtFin Type 1 and BsmtFin Type 2
df_combine.groupby(['BsmtFin Type 1','BsmtFin Type 2'],dropna=False
                  ).size().reset_index().rename(columns={0:'count'})

,BsmtFin Type 1,BsmtFin Type 2,count
0,ALQ,BLQ,30
1,ALQ,GLQ,3
2,ALQ,LwQ,27
3,ALQ,Rec,36
4,ALQ,Unf,333
5,BLQ,ALQ,4
6,BLQ,BLQ,1
7,BLQ,GLQ,10
8,BLQ,LwQ,23
9,BLQ,Rec,34


In [26]:
# Imputing value to BsmtFin Type 2 of row 1147
df_combine.loc[('train',1147),'BsmtFin Type 2'] = 'Rec'

##### 1.3.1.5 Garage Features

We observe 2 non null values in ```Garage Type```, and 158 non null values in ```Garage Cars``` and ```Garage Area```. Based on inspection, at least some of the 158 values are zero, hence we will filter for non zero rows.

In [27]:
df_garage_nan = df_combine[df_combine[col_group['garage']].isna().any(axis=1)][col_group['garage']]
df_garage_nan.info()
#theres one row with nans that have

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 159 entries, ('train', 28) to ('test', 859)
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Garage Type    2 non-null      object 
 1   Garage Yr Blt  0 non-null      float64
 2   Garage Finish  0 non-null      object 
 3   Garage Cars    158 non-null    float64
 4   Garage Area    158 non-null    float64
 5   Garage Qual    0 non-null      object 
 6   Garage Cond    0 non-null      object 
dtypes: float64(3), object(4)
memory usage: 89.9+ KB


In [28]:
#Filter for non zero rows
df_garage_nan[(df_garage_nan['Garage Area']!=0) | 
              (df_garage_nan['Garage Cars']!=0) ]

,,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond
train,1712,Detchd,NaN,NaN,NaN,NaN,NaN,NaN
test,764,Detchd,NaN,NaN,1.0,360.0,NaN,NaN


After filtering for non-zero rows, we find that only two rows remain as shown above. Which are the are two rows with values for Garage Type. Hence it is likely that data is missing. We would impute values for the row from the train dataset, but not the one from test.

We would impute the average value for ```Garage Area``` & ```Garage Cars```, modal values for ```Garage Qual```, ```Garage Cond```, and the same year as the house was built for ```Garage Yr Blt```.

In [29]:
# Impute average for garage_cars & garage_area
df_combine.loc[df_combine['Garage Type']=='Detchd',col_group['garage']].describe()

,Garage Yr Blt,Garage Cars,Garage Area
count,780.000000,781.000000,781.000000
mean,1961.302564,1.549296,419.740077
std,27.079178,0.572166,174.435942
min,1895.000000,1.000000,100.000000
25%,1939.000000,1.000000,280.000000
50%,1962.000000,2.000000,400.000000
75%,1981.000000,2.000000,528.000000
max,2009.000000,5.000000,1488.000000


In [30]:
# We assume that the year built for garage is the same year as the house.
df_combine.loc[('train', 1712), 'Year Built']

1923

In [31]:
garage_col = ['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars', 
              'Garage Area', 'Garage Qual','Garage Cond']
replace_1712 = ['Attchd', 1923, 1923, 1.549296,
                419.740077, 'TA', 'TA']
df_combine.loc[('train',1712),garage_col] = replace_1712

##### 1.3.1.6 External Features

We check Pool QC with Pool Area and we find that the zeros and ```NaN``` are aligned.
For ```Fence``` and ```Misc Features``` there is no other field to cross check so we would take it as no value.

In [32]:
df_combine.groupby(['Pool Area', 'Pool QC'],dropna=False
              ).size().reset_index().rename(columns={0:'count'})

,Pool Area,Pool QC,count
0,0,NaN,2916
1,144,Ex,1
2,228,Ex,1
3,368,TA,1
4,444,TA,1
5,480,Gd,1
6,512,Ex,1
7,519,Fa,1
8,555,Ex,1
9,561,TA,1


#### 1.3.1.7 Process Null/Zero & missing values

We now process the remaining null, zero values as discussed above.

In [33]:
# Generate count of null values by column again
null_series=df_combine.isnull().sum()
display(null_series[null_series>0])
null_cols = null_series[null_series>0].index.tolist()

Alley             2731
Mas Vnr Type        23
Mas Vnr Area        23
Bsmt Qual           80
Bsmt Cond           80
Bsmt Exposure       83
BsmtFin Type 1      80
BsmtFin Type 2      80
Bsmt Unf SF          1
Electrical           1
Bsmt Full Bath       2
Bsmt Half Bath       2
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      158
Garage Finish      158
Garage Qual        158
Garage Cond        158
Pool QC           2916
Fence             2357
Misc Feature      2823
SalePrice          878
bsmt_fin_sf          1
dtype: int64

In [34]:
def convert_nan(df):
    '''Fill NaN values in object dtype columns as string 'none' 
    and int 0 in float64 and int64 columns'''

    to_remove = ['Lot Frontage', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Total Bsmt SF']
    cols = [i for i in null_cols if i not in to_remove]
    
    for i in cols:
        if df[i].dtypes == 'object':
            df[i].fillna('none', inplace=True)
        if df[i].dtypes == 'float64':
            df[i].fillna(0, inplace=True)
        if df[i].dtypes == 'int64':
            df[i].fillna(0, inplace=True)
            
convert_nan(df_combine) 

In [35]:
# Check for null values after data cleaning
null_series=df_combine.isnull().sum()
display(null_series[null_series>0])
null_cols = null_series[null_series>0].index.tolist()

Series([], dtype: int64)

### 1.3.2 Inspect Categorical Variables for Typographical Errors

In this section we take a look at categorical variables and to check if there are any errors in the data compared with the original data documentation.

In [36]:
# Generate counts of categorical values in columns with dtype=object
categories = {key: val.value_counts(dropna=False, normalize=True) 
              for key, val in df_combine.iteritems() 
              if df_combine.dtypes[key] == 'object'}
display(categories)

{'MS Zoning': RL         0.775691
 RM         0.157733
 FV         0.047456
 RH         0.009218
 C (all)    0.008535
 A (agr)    0.000683
 I (all)    0.000683
 Name: MS Zoning, dtype: float64,
 'Street': Pave    0.995903
 Grvl    0.004097
 Name: Street, dtype: float64,
 'Alley': none    0.93240
 Grvl    0.04097
 Pave    0.02663
 Name: Alley, dtype: float64,
 'Lot Shape': Reg    0.634688
 IR1    0.333902
 IR2    0.025947
 IR3    0.005463
 Name: Lot Shape, dtype: float64,
 'Land Contour': Lvl    0.898942
 HLS    0.040970
 Bnk    0.039604
 Low    0.020485
 Name: Land Contour, dtype: float64,
 'Utilities': AllPub    0.998976
 NoSewr    0.000683
 NoSeWa    0.000341
 Name: Utilities, dtype: float64,
 'Lot Config': Inside     0.730283
 Corner     0.174462
 CulDSac    0.061454
 FR2        0.029020
 FR3        0.004780
 Name: Lot Config, dtype: float64,
 'Land Slope': Gtl    0.951861
 Mod    0.042677
 Sev    0.005463
 Name: Land Slope, dtype: float64,
 'Neighborhood': NAmes      0.151246
 Coll

Based on comparison with the original data documentation, we found some errors in ```Exterior 2nd``` which we will fix below. 

In [37]:
def fix_typo_exterior(df):
    typo_dict = {'Wd Shng' : 'WdShing', 
                 'Brk Cmn' : 'BrkComm', 
                 'CmentBd' : 'CemntBd'
                }
    df['Exterior 2nd'].replace(typo_dict, inplace=True)
fix_typo_exterior(df_combine)

## 1.4 Export Cleaned Data

In [38]:
# Save cleaned data file
df_combine.to_csv('./datasets/df_combine_cleaned.csv')